In [0]:
-- we want to build a mini conversion funnel, from /lander-2 to /cart
-- we want to know how many people reach each step, also drop-off rates 
-- for simplicity of the demo, we're looking at /lander-2 traffic only
-- for simplicity of the demo, we're looking at customers who like Mr. Fuzzy only 

-- step 1 : select all pageviews for relevant sessions 
-- step 2 : identify each relevant pageview as the specific funnel step
-- step 3 : create the session-level conversion funnel view
-- step 4 : aggregate the data to assess funnel performance 

select 
  ws.website_session_id,
  wp.pageview_url,
  wp.created_at as pageview_created_at,
  case when wp.pageview_url = '/lander-2' then 1 else 0 end as products_page,
  case when wp.pageview_url = '/the-original-mr-fuzzy' then 1 else 0 end as myfuzzy_page,
  case when wp.pageview_url = '/cart' then 1 else 0 end as cart_page
from website_sessions ws
left join website_pageviews wp
on 
  ws.website_session_id = wp.website_session_id
where 
  ws.created_at between '2014-01-01' and '2014-02-01'
  and wp.pageview_url in ('/lander-2', '/the-original-mr-fuzzy', '/cart', '/products')
order by 
  ws.website_session_id,
  wp.created_at


  ---------------------------SUB QUERY----------------------
create temporary table session_level_made_it_flags_demo
select 
  website_session_id,
  max(products_page) as product_made_it,
  max(myfuzzy_page) as myfuzzy_made_it,
  max(cart_page) as cart_made_it
  -- ,max(shipping_page) as shipping_made_it,
  -- max(billing_page) as billing_made_it,
  -- max(thankyou_page) as thankyou_made_it
  from (
    select 
      ws.website_session_id,
      wp.pageview_url,
      wp.created_at as pageview_created_at,
      case when wp.pageview_url = '/lander-2' then 1 else 0 end as products_page,
      case when wp.pageview_url = '/the-original-mr-fuzzy' then 1 else 0 end as myfuzzy_page,
      case when wp.pageview_url = '/cart' then 1 else 0 end as cart_page
    from website_sessions ws
    left join website_pageviews wp
    on 
      ws.website_session_id = wp.website_session_id
    where 
      ws.created_at between '2014-01-01' and '2014-02-01'
      and wp.pageview_url in ('/lander-2', '/the-original-mr-fuzzy', '/cart', '/products')
    order by 
      ws.website_session_id,
      wp.created_at
) AS pageview_level
group by 1

---------------------------------

select 
  count(distinct website_session_id) as sessions,
  count(distinct case when product_made_it = 1 then website_session_id end) as products_page,
  count(distinct case when myfuzzy_made_it = 1 then website_session_id end) as myfuzzy_page,
  count(distinct case when cart_made_it = 1 then website_session_id end) as cart_page
from 
  session_level_made_it_flags_demo

-----------------------------------------

select 
  count(distinct website_session_id) as sessions,
  -- count(distinct case when product_made_it = 1 then website_session_id end) as products_page,
  count(distinct case when product_made_it = 1 then website_session_id end) / count(distinct website_session_id)  as clickthroughrate_products_page,

  -- count(distinct case when myfuzzy_made_it = 1 then website_session_id end) as myfuzzy_page,
  count(distinct case when myfuzzy_made_it = 1 then website_session_id end) / count(distinct website_session_id)  as clickthroughrate_mrfuzzy_page,

  -- count(distinct case when cart_made_it = 1 then website_session_id end) as cart_page,
  count(distinct case when  cart_made_it = 1 then website_session_id end) / count(distinct website_session_id)  as clickthroughrate_cart_page
from 
  session_level_made_it_flags_demo
